In [1]:
import numpy as np
import argparse
import random
import time
import cv2
import os
from pathlib import Path
import matplotlib as plt
%matplotlib inline

In [2]:
path = Path('/Users/sandeep/Desktop/dataandmodles/models/teamDetection/mask-rcnn/mask-rcnn-coco')
args = {'img':'/Users/sandeep/Desktop/dataandmodles/data/pz3Pointer.png',
       'img2':'/Users/sandeep/Desktop/dataandmodles/data/lakers.png',
       'img3':'/Users/sandeep/Desktop/dataandmodles/data/pz3Pointer2.png'}
RED_COLOR = np.array([255, 0, 0]) 
BLACK_COLOR = np.array([255, 255, 255]) 

In [3]:
# Load classes
classes_file = "/Users/sandeep/Desktop/MaskRCNNopencv/mscoco_labels.names"
text_graph = '/Users/sandeep/Desktop/dataandmodles/models/teamDetection/Mask_RCNN/mask_rcnn_inception_v2_coco_2018_01_28.pbtxt'
model_weights = '/Users/sandeep/Desktop/dataandmodles/models/teamDetection/Mask_RCNN/mask_rcnn_inception_v2_coco_2018_01_28/frozen_inference_graph.pb'

# load our Mask R-CNN trained on the COCO dataset (90 classes) from disk
print("[INFO] loading Mask R-CNN from disk...")
net = cv2.dnn.readNetFromTensorflow(model_weights, text_graph)

[INFO] loading Mask R-CNN from disk...


In [3]:
#Load images 
image = cv2.imread(args['img3'])
(H, W) = image.shape[:2]
print("[INFO] image size: {}x{} pixels".format(W, H))

[INFO] image size: 1242x676 pixels


In [5]:

# construct a blob from the input image and then perform a forward
# pass of the Mask R-CNN, giving us (1) the bounding box coordinates
# of the objects in the image along with (2) the pixel-wise segmentation
# for each specific object
blob = cv2.dnn.blobFromImage(image, swapRB=True, crop=False)
net.setInput(blob)
start = time.time()
(boxes, masks) = net.forward(["detection_out_final", "detection_masks"])
end = time.time()
# show timing information and volume information on Mask R-CNN
print("[INFO] Mask R-CNN took {:.6f} seconds".format(end - start))
print("[INFO] boxes shape: {}".format(boxes.shape))
print("[INFO] boxes size: {}".format(boxes.size))
print("[INFO] masks shape: {}".format(masks.shape))

[INFO] Mask R-CNN took 2.174696 seconds
[INFO] boxes shape: (1, 1, 100, 7)
[INFO] boxes size: 700
[INFO] masks shape: (100, 90, 15, 15)


In [6]:
# loop over the number of detected objects
for i in range(0, boxes.shape[2]):

	# extract the class ID of the detection along with the confidence
	# (i.e., probability) associated with the prediction
	classID = int(boxes[0, 0, i, 1])
	confidence = boxes[0, 0, i, 2]

	# filter out weak predictions by ensuring the detected probability
	# is greater than the minimum probability
	if confidence > 0.7:
		# clone our original image so we can draw on it
		# clone = image.copy()

		# scale the bounding box coordinates back relative to the
		# size of the image and then compute the width and the height
		# of the bounding box
		box = boxes[0, 0, i, 3:7] * np.array([W, H, W, H])
		(startX, startY, endX, endY) = box.astype("int")
		boxW = endX - startX
		boxH = endY - startY

		# extract the pixel-wise segmentation for the object, resize
		# the mask such that it's the same dimensions of the bounding
		# box, and then finally threshold to create a *binary* mask
		mask = masks[i, classID]
		mask = cv2.resize(mask, (boxW, boxH), interpolation = cv2.INTER_NEAREST)
		mask = (mask > 0.3)

		# extract the ROI of the image
		roi = image[startY:endY, startX:endX]
		print(roi.shape)
		# now, extract *only* the masked region of the ROI by passing in the boolean mask array as our slice condition
		roi = roi[mask]
		print(roi.shape)
        

		# Red will be used to visualize this particular instance segmentation 
		# then create a transparent overlay by blending the randomly selected color with the ROI
		blended = ((0.4 * RED_COLOR) + (0.6 * roi)).astype("uint8")

		# store the blended ROI in the original image
		image[startY:endY, startX:endX][mask] = blended

		# draw the bounding box of the instance on the image
		cv2.rectangle(image, (startX, startY), (endX, endY), (255,255,255), 2)

		# draw the predicted label and associated probability of the instance segmentation on the image
		text = "{}: {:.4f}".format("Person", confidence)
		cv2.putText(image, text, (startX, startY - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 2)
#cv2.imwri('/Users/sandeep/Desktop/dataandmodles/data/ff.png', image)     

(229, 64, 3)
(7489, 3)
(222, 58, 3)
(6691, 3)
(163, 60, 3)
(5268, 3)
(242, 70, 3)
(8988, 3)
(193, 78, 3)
(9134, 3)
(138, 48, 3)
(4414, 3)
(211, 117, 3)
(9674, 3)
(85, 85, 3)
(3565, 3)
(101, 50, 3)
(3351, 3)


In [9]:
RED_COLOR.shape

(3,)

In [38]:
roi

array([[255, 255, 255],
       [255, 255, 255],
       [255, 255, 255],
       ...,
       [255, 255, 255],
       [255, 255, 255],
       [255, 255, 255]], dtype=uint8)

In [14]:
mask.shape

(226, 62)

In [29]:
image[startY:endY, startX:endX].shape

(134, 46, 3)

In [31]:
mask

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

In [32]:
roi.shape

(4647, 3)

In [34]:
blended.shape

(4647, 3)

In [75]:
image[startY:endY, startX:endX][mask]
#np.unique(image[startY:endY, startX:endX][mask])

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208,
       209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221,
       222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234,
       235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247,
       248, 249, 250, 251, 252, 253, 254, 255], dtype=uint8)

In [58]:
image[startY:endY, startX:endX].shape

(134, 46, 3)

In [45]:
mask.shape

(134, 46)

In [56]:
roi

array([[255, 255, 255],
       [255, 255, 255],
       [255, 255, 255],
       ...,
       [255, 255, 255],
       [255, 255, 255],
       [255, 255, 255]], dtype=uint8)

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

In [68]:
a= np.array([[255,255,255], [0,0,0],[3,3,3]])
a.shape

(3, 3)

In [67]:
m = np.array([[True,True,True],
              [False,False,False], 
              [True,True,True]])
m.shape

(3, 3)

In [73]:
mask[0].shape

(46,)

In [79]:
cv2.cvtColor(np.array([255,255,255]), cv2.COLOR_BGR2HSV)

error: OpenCV(4.2.0) /Users/travis/build/skvark/opencv-python/opencv/modules/imgproc/src/color.simd_helpers.hpp:92: error: (-2:Unspecified error) in function 'cv::impl::(anonymous namespace)::CvtHelper<cv::impl::(anonymous namespace)::Set<3, 4, -1>, cv::impl::(anonymous namespace)::Set<3, -1, -1>, cv::impl::(anonymous namespace)::Set<0, 5, -1>, cv::impl::(anonymous namespace)::SizePolicy::NONE>::CvtHelper(cv::InputArray, cv::OutputArray, int) [VScn = cv::impl::(anonymous namespace)::Set<3, 4, -1>, VDcn = cv::impl::(anonymous namespace)::Set<3, -1, -1>, VDepth = cv::impl::(anonymous namespace)::Set<0, 5, -1>, sizePolicy = cv::impl::(anonymous namespace)::SizePolicy::NONE]'
> Invalid number of channels in input image:
>     'VScn::contains(scn)'
> where
>     'scn' is 1


In [87]:
green = np.uint8([[[255,255,255 ]]])
hsv_green = cv2.cvtColor(green,cv2.COLOR_BGR2HSV)

In [88]:
hsv_green

array([[[  0,   0, 255]]], dtype=uint8)

In [82]:
green.shape

(1, 1, 3)

In [83]:
image[startY:endY, startX:endX][mask].shape

(4647, 3)

In [85]:
h = np.reshape(image[startY:endY, startX:endX][mask], (1,4647,3))

In [86]:
cv2.cvtColor(h,cv2.COLOR_BGR2HSV)

array([[[  0,   0, 255],
        [  0,   0, 255],
        [  0,   0, 255],
        ...,
        [  0,   0, 255],
        [  0,   0, 255],
        [  0,   0, 255]]], dtype=uint8)

In [ ]:
cv2.imshow('aa',image)
cv2.waitKey(0)

In [ ]:
import cv2
import sys

In [4]:
cv2.__version__

'4.1.2'

In [4]:
sys.path

['/Users/sandeep/Desktop/FinalYearProject/Python Files/teamDetection',
 '/Users/sandeep/Desktop/FinalYearProject/Python Files/teamDetection',
 '',
 '/System/Library/Frameworks/Python.framework/Versions/2.6/lib/python2.6/site-packages',
 '/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages',
 '/Users/sandeep/opt/anaconda3/envs/cv/lib/python37.zip',
 '/Users/sandeep/opt/anaconda3/envs/cv/lib/python3.7',
 '/Users/sandeep/opt/anaconda3/envs/cv/lib/python3.7/lib-dynload',
 '/Users/sandeep/.local/lib/python3.7/site-packages',
 '/Users/sandeep/opt/anaconda3/envs/cv/lib/python3.7/site-packages',
 '/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/IPython/extensions',
 '/Users/sandeep/.ipython']

In [3]:
cv2.__path__

['/Users/sandeep/opt/anaconda3/envs/cv/lib/python3.7/site-packages/cv2']